# Requirements and Imports

Requirements installation

In [ ]:
%pip install pydot
%pip install cupy-cuda11x
%pip install pm4py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 MB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.2 MB/s eta 0:00:00
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26097 sha256=0ac0e3463c35c6ff02196a6ecc3d306a48ca06d7e369e980115174f742a85bc4
  Stored in directory: /root/.cache/pip/wheels/fa/80/8c/43488a924a046b733b64de3fac99252674c892a4c3801c0a61
Successfully built intervaltree


We mount Google Drive for files access

In [ ]:
# Drive mounting

from google.colab import files
from google.colab import drive


directory = '/content/problems'

!rm {directory}/*

!rm *

print("Please Upload the dot formatted declare file")
uploaded = files.upload()

for file_name in uploaded.keys():
    print(f'Uploaded file: {file_name}')


drive.mount('/content/drive')


rm: cannot remove '/content/problems/*': No such file or directory
rm: cannot remove 'sample_data': Is a directory
Please Upload the dot formatted declare file


Saving declare.dot to declare.dot
Uploaded file: declare.dot
Mounted at /content/drive


We make the required imports


In [ ]:
import pydot
import random
from datetime import datetime, timedelta
import os
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.log.exporter.xes import exporter as xes_exporter
import os
import shutil

/usr/local/lib/python3.10/dist-packages/cudf/utils/_ptxcompiler.py:64: UserWarning: Error getting driver and runtime versions:

stdout:



stderr:

Traceback (most recent call last):
  File "<string>", line 4, in <module>
  File "/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/driver.py", line 295, in __getattr__
    raise CudaSupportError("Error at driver init: \n%s:" %
numba.cuda.cudadrv.error.CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:


Not patching Numba
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.10/dist-packages/cudf/utils/gpu_utils.py:62: UserWarning: Failed to dlopen libcuda.so.1
  warnings.warn(str(e))
/usr/local/lib/python3.10/dist-packages/cupy/_environment.py:540: UserWarning: 
--------------------------------------------------------------------------------

  C

# Pddl Translator

We set the required global parameters

In [ ]:
# let the user define this section
xes_file = "drive/MyDrive/TESI/SRC/simulation_logs.xes"
# constraint files in dot format
dot_files = ["declare.dot"]
domain_name = "traceAlignment"
problem_limit = 4 # if you want to parse problems until a bound is reached
problem_automata = 1 # number of automata for the dot files if more than one
problem_automata_list = [1,2,5,9] # if you want to sample traces
lifecycles = ["assign","start","complete"] # let the user define its own lifecycles
lifecycle_states = ["init_state","assigned_state","started_state","completed_state","sink_state"]
resources = ["Susi", "John", "Eric"] # let the user define its own resources
# log traces tags
trace_id_tag = "concept:name"
event_id_tag = "concept:name"
activity_tag = "Activity"
lifecycle_tag = "lifecycle:transition"
org_tag = "org:resource"
resource_tag = "Resource"
timestamp_tag = "time:timestamp"

Function to write the pddl problem to a .pddl file

In [ ]:
def write_pddl_problem(output_file, problem_name, initial_state, objects, goal_state):
    """
    Function to write the pddl problem with content settled by parameters in input.

    Parameters:
        output_file (file name) : The output file name.
        problem_name (file name): The pddl problem name.
        initial_state (list): The lines representing the initial state of the pddl problem.
        objects (dict): A dictionary representing the objects of the pddl problem.
        goal_state (list): The lines representing the goal state of the pddl problem.

    """
    directory = '/content/problems'

    file_path = os.path.join(directory, output_file)

    if not os.path.exists(directory):
        os.makedirs(directory)

    with open(file_path, 'w') as f:
        f.write(f"(define (problem {problem_name})\n")
        f.write(f"\t(:domain {domain_name})\n\n")

        # Objects section
        f.write("\t(:objects\n")
        for obj_type, obj_list in objects.items():
            f.write(f"\t\t{' '.join(obj_list)} - {obj_type}\n")
        f.write("\t)\n\n")

        # Initial state section
        f.write("\t(:init\n")
        for predicate in initial_state:
            f.write(f"\t\t{predicate}\n")
        f.write("\t)\n\n")

        # Goal state section
        f.write("\t(:goal\n")
        f.write(f"\t\t(and\n")
        for predicate in goal_state:
            f.write(f"\t\t\t{predicate}\n")
        f.write("\t\t)\n")
        f.write("\t)\n")
        f.write("\t(:metric minimize (total-cost))\n")
        f.write(")\n")
    print("Writed file : ",file_path)

Function to generate automata from a .dot file

In [ ]:
def generate_formula(initial_state,objects,goal_state,found_activities):

   for automaton_index in range(0,problem_automata):
      # Parse the .dot file and extract graph structure
      graph = pydot.graph_from_dot_file(dot_files[automaton_index])[0]
      if automaton_index == 0:
        # Iterate over each automaton
        automaton = {"states": [], "transitions": []}

        # Extract states and transitions from graph
        for node in graph.get_nodes():
            state_name = node.get_name().strip('"')
            state_shape = node.get_shape().strip('"')
            if state_name:
              if state_name != "initial":
                automaton["states"].append(f"s{state_name}")
                if state_shape == "doublecircle":
                  initial_state.append(f"(final_state s{state_name})")
                  goal_state.append(f"(cur_state s{state_name})")



        for edge in graph.get_edges():
            source = edge.get_source().strip('"')
            target = edge.get_destination().strip('"')
            original_labels_string = edge.get_label()
            original_label= edge.get_label()
            label = edge.get_label()  # Get the label of the edge (arc)
            if label != None :
              label = label.replace('"','')
              for lifecycle in lifecycles:
                  if label.find(f"_{lifecycle}") != -1:
                      label = label.replace(f"_{lifecycle}", "")
                      break
              if label.find('\\n') != -1 :
                original_labels_string = original_labels_string.replace('"','')
                original_labels_string_array = original_labels_string.split('\\n')
                labels = label.split('\\n')
                length = len(labels)
                activities = []
                for i in range(0,length):
                  activities.append(labels[i])
                for act in activities :
                  if act not in found_activities :
                    initial_state.append(f"(cur_lifecycle_state {act} {lifecycle_states[0]})") # this is always the [0]
                    initial_state.append(f"(final_lifecycle_state {act} {lifecycle_states[-2]})") # this is always the n-2 element of lifecycles
                    # this for each lifecycles[i]
                    for l in lifecycles:
                      initial_state.append(f"(lifecycle_activity_of {act} {act}_{l})")
                    # this block for each lifecycles i from 0 to n-2 i have lifecycle_states[i] act_lifecycle[i] lifecycle_states[i+1] and lifecycle_states[i] act_lifecycle[j!=i] lifecycle_states[n-1]
                    for i in range(len(lifecycle_states)) :
                      if i != len(lifecycle_states)- 1 :
                        if i!= len(lifecycle_states) - 2:
                          for j in range(len(lifecycles)):
                            if j == i:
                              initial_state.append(f"(lifecycle {lifecycle_states[i]} {act}_{lifecycles[j]} {lifecycle_states[i+1]})")
                            else:
                              initial_state.append(f"(lifecycle {lifecycle_states[i]} {act}_{lifecycles[j]} {lifecycle_states[-1]})") # comment if no sink states wanted
                        else:
                          for j in range(len(lifecycles)):
                            if j == 0:
                              initial_state.append(f"(lifecycle {lifecycle_states[i]} {act}_{lifecycles[j]} {lifecycle_states[j]})")
                            else:
                              initial_state.append(f"(lifecycle {lifecycle_states[i]} {act}_{lifecycles[j]} {lifecycle_states[-1]})") # comment if no sink states wanted

                    goal_state.append(f"(cur_lifecycle_state {act} {lifecycle_states[-2]})") # this is always the n-2 element of lifecycles
                    objects["activity"].append(act)

                    # this for each lifecycles
                    for l in lifecycles:
                      objects["lifecycle_activity"].append(f"{act}_{l}")


                    found_activities.append(act)
                for act_name in original_labels_string_array :
                  if source and target and act_name:
                    if source != target :
                      if source != "initial" :
                        automaton["transitions"].append((source, target, act_name))
                      else:
                        initial_state.append(f"(cur_state s{target})")
              else :
                act= label
                if act not in found_activities :
                    initial_state.append(f"(cur_lifecycle_state {act} {lifecycle_states[0]})") # this is always the [0]
                    initial_state.append(f"(final_lifecycle_state {act} {lifecycle_states[-2]})") # this is always the n-2 element of lifecycles
                    # this for each lifecycles[i]

                    # this block for each lifecycles i from 0 to n-2 i have lifecycle_states[i] act_lifecycle[i] lifecycle_states[i+1] and lifecycle_states[i] act_lifecycle[j!=i] lifecycle_states[n-1]
                    for i in range(len(lifecycle_states)) :
                      if i != len(lifecycle_states)- 1 :
                        if i!= len(lifecycle_states) - 2:
                          for j in range(len(lifecycles)):
                            if j == i:
                              initial_state.append(f"(lifecycle {lifecycle_states[i]} {act}_{lifecycles[j]} {lifecycle_states[i+1]})")
                            else:
                              initial_state.append(f"(lifecycle {lifecycle_states[i]} {act}_{lifecycles[j]} {lifecycle_states[-1]})") # comment if no sink states wanted
                        else:
                          for j in range(len(lifecycles)):
                            if j == 0:
                              initial_state.append(f"(lifecycle {lifecycle_states[i]} {act}_{lifecycles[j]} {lifecycle_states[j]})")
                            else:
                              initial_state.append(f"(lifecycle {lifecycle_states[i]} {act}_{lifecycles[j]} {lifecycle_states[-1]})") # comment if no sink states wanted

                    goal_state.append(f"(cur_lifecycle_state {act} {lifecycle_states[-2]})") # this is always the n-2 element of lifecycles
                    objects["activity"].append(act)

                    # this for each lifecycles
                    for l in lifecycles:
                      objects["lifecycle_activity"].append(f"{act}_{l}")
                      initial_state.append(f"(lifecycle_activity_of {act} {act}_{l})")



                    found_activities.append(act)
                if source and target and original_label:
                 if source != target :
                   if source != "initial" :
                     automaton["transitions"].append((source, target, original_label))
                   else:
                     initial_state.append(f"(cur_state s{target})")


        # Extract from declare model to formula automata
        start = True
        for state in automaton["states"]:
          if start == True :
            initial_state.append(f"(cur_state {state})")
            start = False
          objects["automaton_state"].append(state)
        for source, target, label in automaton["transitions"]:
            label = label.replace('"','')
            initial_state.append(f"(automaton s{source} {label} s{target})")

# Execution

In this section we have the main functions to execute the overall program

In [ ]:
def generate():
    """
    Main Function to generate pddl problems from:
    1) The xes formatted log imported with pm4py.
    2) The dot formatted declare model.

    The function can inject noise in the traces and can cat the log traces.

    Parameters: None

    Returns:
        A string stating the result of the generation or None if something went wrong during the process.
    """

    # Parse the XES log
    log = xes_importer.apply(xes_file)
    if log is None:
        return log

    num = 0
    # Extract activities and lifecycle from xes to lifecycle and trace automata
    for trace_index, trace in enumerate(log):
        num = num + 1
        if num in problem_automata_list:
            # clean structures
            print("New trace", trace.attributes[trace_id_tag])
            trace_id = trace.attributes[trace_id_tag].replace(" ", "_")
            message = ""
            problem_name = f"trace_Alignment_Problem_{trace_id}{message}.pddl"
            objects = {
                "trace_state": [],
                "automaton_state": [],
                "activity": [],
                "lifecycle_activity": []
            }
            initial_state = []
            goal_state = []
            found_activities = []
            generate_formula(initial_state, objects, goal_state, found_activities)

            i = 0
            cur_activity = "init"
            initial_state.append(f"(cur_state t{i})")

            for event_index, event in enumerate(trace):
                if cur_activity != event[event_id_tag].replace(" ", "_"):
                    # new activity found
                    print("Old activity: ", cur_activity, ", New activity: ", event[event_id_tag].replace(" ", "_"))
                    cur_activity = event[event_id_tag].replace(" ", "_")
                    # setting lifecycle for the new activity not encountered before
                    if cur_activity not in found_activities:
                        objects["trace_state"].append(f"t{i}")
                        initial_state.append(f"(trace t{i} {cur_activity}_{lifecycles[0]} t{i+1})")
                        i = i + 1
                        initial_state.append(f"(cur_lifecycle_state {cur_activity} {lifecycle_states[0]})")
                        initial_state.append(f"(final_lifecycle_state {cur_activity} {lifecycle_states[-2]})")

                        goal_state.append(f"(cur_lifecycle_state {cur_activity} {lifecycle_states[-2]})")
                        objects["activity"].append(cur_activity)
                        for l in lifecycles:
                            objects["lifecycle_activity"].append(f"{cur_activity}_{l}")
                            initial_state.append(f"(lifecycle_activity_of {cur_activity} {cur_activity}_{l})")

                        for k in range(len(lifecycle_states)):
                            if k != len(lifecycle_states) - 1:
                                if k != len(lifecycle_states) - 2:
                                    for j in range(len(lifecycles)):
                                        if j == k:
                                            initial_state.append(f"(lifecycle {lifecycle_states[k]} {cur_activity}_{lifecycles[j]} {lifecycle_states[k+1]})")
                                        else:
                                            initial_state.append(f"(lifecycle {lifecycle_states[k]} {cur_activity}_{lifecycles[j]} {lifecycle_states[-1]})") # comment if no sink states wanted
                                else:
                                    for j in range(len(lifecycles)):
                                        if j == 0:
                                            initial_state.append(f"(lifecycle {lifecycle_states[k]} {cur_activity}_{lifecycles[j]} {lifecycle_states[j]})")
                                        else:
                                            initial_state.append(f"(lifecycle {lifecycle_states[k]} {cur_activity}_{lifecycles[j]} {lifecycle_states[-1]})") # comment if no sink states wanted

                        found_activities.append(cur_activity)

                # lifecycle activity name
                cur_lifecycle = event[lifecycle_tag]

                # adding transition state
                objects["trace_state"].append(f"t{i}")
                initial_state.append(f"(trace t{i} {cur_activity}_{cur_lifecycle} t{i+1})")
                i = i + 1

            objects["trace_state"].append(f"t{i}")
            initial_state.append(f"(final_state t{i})")
            goal_state.append(f"(cur_state t{i})")

            # Create PDDL problem file
            write_pddl_problem(problem_name, problem_name, initial_state, objects, goal_state)

    return "Completed"

if __name__ == "__main__":

    print("\n-----WELCOME TO THE PDDL TRANSALTOR-----\n")

    result = generate()
    if result == "Completed":
        print("Success: problems generated")
    if result is None:
        print("Error: something went wrong")

    print("\n-----THANK YOU FOR USING PDDL TRANSALTOR-----\n")



-----WELCOME TO THE PDDL TRANSALTOR-----



/usr/local/lib/python3.10/dist-packages/pm4py/util/dt_parsing/parser.py:82: UserWarning: ISO8601 strings are not fully supported with strpfromiso for Python versions below 3.11
  warnings.warn(


parsing log, completed traces ::   0%|          | 0/40 [00:00<?, ?it/s]

New trace 30
Old activity:  init , New activity:  Credit_application_received
Old activity:  Credit_application_received , New activity:  Check_credit_history
Old activity:  Check_credit_history , New activity:  Check_income_sources
Old activity:  Check_income_sources , New activity:  Check_credit_history
Old activity:  Check_credit_history , New activity:  Check_income_sources
Old activity:  Check_income_sources , New activity:  Assess_application
Old activity:  Assess_application , New activity:  Make_credit_offer
Old activity:  Make_credit_offer , New activity:  Credit_application_processed
Writed file :  /content/problems/trace_Alignment_Problem_30.pddl
New trace 40
Old activity:  init , New activity:  Credit_application_received
Old activity:  Credit_application_received , New activity:  Check_credit_history
Old activity:  Check_credit_history , New activity:  Check_income_sources
Old activity:  Check_income_sources , New activity:  Check_credit_history
Old activity:  Check_credit

# Problems Exports

In this section we export as a zip the problems generated previously

In [ ]:
current_dir = '/content/problems'

temp_dir = '/content/temp_zip'
os.makedirs(temp_dir, exist_ok=True)

files_in_dir = os.listdir(current_dir)

for file_name in files_in_dir:
    file_path = os.path.join(current_dir, file_name)
    if os.path.isfile(file_path):
        shutil.copy(file_path, temp_dir)

shutil.make_archive('/content/problems', 'zip', temp_dir)

files.download('/content/problems.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>